In [2]:
import pickle
import os
import faiss
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [5]:
BASE_PATH = 'C:\\Users\\vladp\\Downloads'
embeddings_file = 'list_256_emb_dump.pkl'
labels_file = 'labels.pkl'
with open(os.path.join(BASE_PATH,embeddings_file),'rb') as f:
    embeddings = pickle.load(f)

with open(os.path.join(BASE_PATH,labels_file),'rb') as f:
    labels = pickle.load(f)

In [89]:
import timm
import torch

In [7]:
embeddings.shape

(3301, 256)

In [6]:
index = faiss.IndexFlatL2(full_list.shape[1])

In [7]:
index.add(full_list)

In [10]:
full_list[0]

array([ 0.84039706,  0.61968762, -0.26464745, ...,  0.37435132,
       -0.22467731, -0.26664886])

In [13]:
d = full_list.shape[1]

In [14]:
nlist = 50  # how many cells
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

In [15]:
index.is_trained

False

In [16]:
index.train(full_list)

In [17]:
index.add(full_list)

In [21]:
D, I = index.search(test, 4)

In [22]:
D

array([[    0.   , 30577.441, 48894.36 , 49319.617]], dtype=float32)

In [23]:
I

array([[  0,  15, 474, 922]], dtype=int64)

In [19]:


nbrs = NearestNeighbors(n_neighbors=9, algorithm='ball_tree').fit(embeddings)

array(['La Conquête de la Franche-Comté par le mois de juin',
       'La Conquête de la Franche-Comté par le mois de juin',
       'La Conquête de la Franche-Comté par le mois de juin',
       'La Conquête de la Franche-Comté par le mois de juin',
       'La Conquête de la Franche-Comté par le mois de juin',
       'La Conquête de la Franche-Comté par le mois de juin',
       'La Conquête de la Franche-Comté par le mois de juin',
       'La Conquête de la Franche-Comté par le mois de juin',
       'La Conquête de la Franche-Comté par le mois de juin',
       'La Conquête de la Franche-Comté par le mois de juin',
       'La Conquête de la Franche-Comté par le mois de juin',
       'La Conquête de la Franche-Comté par le mois de juin',
       'La Conquête de la Franche-Comté par le mois de juin',
       'La Conquête de la Franche-Comté par le mois de juin',
       'La Conquête de la Franche-Comté par le mois de juin'],
      dtype='<U57')

In [49]:
D,I

(array([[  0.        ,  74.81604654,  86.24208872, 110.92703408,
         113.23342934, 117.98597112, 131.29108469, 144.49378774,
         159.26165694]]),
 array([[ 205,  202,  214, 2431, 2437,  211,  208, 1821, 3258]],
       dtype=int64))

In [70]:
labels[215:226]

array(['Maxime aux bas verts', 'Maxime aux bas verts',
       'Maxime aux bas verts', 'Maxime aux bas verts',
       'Maxime aux bas verts', 'Maxime aux bas verts',
       'Maxime aux bas verts', 'Maxime aux bas verts',
       'Maxime aux bas verts', 'Maxime aux bas verts',
       'Maxime aux bas verts'], dtype='<U57')

In [8]:
import pandas as pd
df = pd.read_csv(os.path.join(BASE_PATH,'images.csv'))

In [113]:
#test = np.reshape(embeddings[225],(1,-1))
D,I = nbrs.kneighbors(res_emb)

In [114]:
D,I

(array([[657.54640143, 683.7392696 , 684.03659703, 684.25226231,
         690.7331279 , 706.39705583, 707.2422513 , 707.32846848,
         710.81232513]]),
 array([[3181,  249,  238,  237, 3189, 3182, 3194,  239,  256]],
       dtype=int64))

In [76]:

dft

,D,I,label
0,0.000000,225.0,Maxime aux bas verts
1,136.028790,217.0,Maxime aux bas verts
2,150.781031,219.0,Maxime aux bas verts
3,190.247989,221.0,Maxime aux bas verts
4,190.326088,224.0,Maxime aux bas verts
5,198.366710,215.0,Maxime aux bas verts
6,223.668085,220.0,Maxime aux bas verts
7,241.452093,222.0,Maxime aux bas verts
8,275.893418,218.0,Maxime aux bas verts


In [64]:
dft

,D,I,label
0,0.000000,205.0,La Conquête de la Franche-Comté par le mois de...
1,74.816047,202.0,La Conquête de la Franche-Comté par le mois de...
2,86.242089,214.0,La Conquête de la Franche-Comté par le mois de...
3,110.927034,2431.0,Tête de turc
4,113.233429,2437.0,Tête de turc
5,117.985971,211.0,La Conquête de la Franche-Comté par le mois de...
6,131.291085,208.0,La Conquête de la Franche-Comté par le mois de...
7,144.493788,1821.0,Le Cheval blanc
8,159.261657,3258.0,Explosion lyrique n°8


In [91]:
model = torch.load(os.path.join(BASE_PATH,'model_256_emb_dump.pth'),map_location=torch.device('cpu'))

In [92]:
import torchvision.transforms as transforms
from PIL import Image

img_raw = Image.open(os.path.join(BASE_PATH,'test_composition1t.png'))


transformations = transforms.Compose([
    transforms.Resize([256,256]),
    transforms.ToTensor(),
])

test_im = transformations(img_raw)

In [96]:
test_im = test_im.unsqueeze(0)

In [97]:
test_im.shape

torch.Size([1, 3, 256, 256])

In [115]:
dft = pd.DataFrame(np.concatenate([D,I]).T,columns=['D','I'])
dft['label'] = labels[dft['I'].astype(int)]
dft.groupby('label').mean().sort_values('D')

,D,I
label,,
La Rue,690.479709,3186.5
"Traits, plans et profondeur 1",694.033785,243.8


In [100]:
base_model = timm.create_model('tf_efficientnetv2_s',pretrained=True)
base_model.eval()

EfficientNet(
  (conv_stem): Conv2dSame(3, 24, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNormAct2d(
    24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): ConvBnAct(
        (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (drop_path): Identity()
      )
      (1): ConvBnAct(
        (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (drop_path): Identity()
      )
    )
    (1): Sequential(
   

In [101]:
import torch.nn as nn
import torch.optim as optim

class Network(nn.Module):
    def __init__(self, emb_dim=256):
        super(Network, self).__init__()
        self.base_model = base_model.forward_features
        
        self.fc = nn.Sequential(
            nn.Linear(1280*8*8, 2048),
            nn.PReLU(),
            nn.Linear(2048, emb_dim)
        )
        
    def forward(self, x):
        x = self.base_model(x)
        x = x.view(-1, 1280*8*8)
        x = self.fc(x)
        # x = nn.functional.normalize(x)
        return x

In [105]:
model = Network(256)
model.to('cpu')
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.TripletMarginLoss(margin=1.0, p=2)

In [109]:
model.load_state_dict(torch.load(os.path.join(BASE_PATH,'model_new_state_dict.pth'),map_location=torch.device('cpu')))

<All keys matched successfully>

In [110]:
model.eval()

Network(
  (fc): Sequential(
    (0): Linear(in_features=81920, out_features=2048, bias=True)
    (1): PReLU(num_parameters=1)
    (2): Linear(in_features=2048, out_features=256, bias=True)
  )
)

In [111]:
with torch.no_grad():
    res_emb = model(test_im)

In [112]:
res_emb.shape

torch.Size([1, 256])

In [79]:
dft.groupby('label').mean().sort_values('D').index[0]

'Maxime aux bas verts'

In [12]:
labels.shape

(3301,)

In [13]:
df['class'][:5]

0    04.05.64
1    04.05.64
2    04.05.64
3    04.05.64
4    04.05.64
Name: class, dtype: object

In [17]:
np.unique(labels)[np.unique(labels)!=np.unique(df['class'])]

array(['Chapeau de paille?', 'Meurtre n°20:1,',
       'Objets usuels, pour Vincent Van Gogh?', 'Où es-tu? Que fais-tu?,',
       'Spatiodynamique 26 (dite "Gigogne")'], dtype='<U57')

In [18]:
np.unique(df['class'])[np.unique(labels)!=np.unique(df['class'])]

array(['Chapeau de paille_', 'Meurtre n°20_1,',
       'Objets usuels, pour Vincent Van Gogh_', 'Où es-tu_ Que fais-tu_,',
       'Spatiodynamique 26 (dite _Gigogne_)'], dtype=object)

In [29]:
df[100:].head()

,class,image
100,Rythme,"Фото 17.04.2023, 16 28 12.jpg"
101,Rythme,"Фото 17.04.2023, 16 28 00.jpg"
102,Rythme,"Фото 17.04.2023, 16 28 01.jpg"
103,Rythme,"Фото 17.04.2023, 16 28 03.jpg"
104,Rythme,"Фото 17.04.2023, 16 28 06.jpg"


In [36]:
I.shape

(1, 9)

In [83]:
labels[labels=='04.05.64']

array(['04.05.64', '04.05.64', '04.05.64', '04.05.64', '04.05.64',
       '04.05.64', '04.05.64', '04.05.64', '04.05.64', '04.05.64',
       '04.05.64', '04.05.64', '04.05.64', '04.05.64', '04.05.64',
       '04.05.64', '04.05.64', '04.05.64'], dtype='<U57')

In [84]:
labels_df = pd.DataFrame(labels,columns=['class'])

In [86]:
labels_df[labels_df['class']=='04.05.64'].index

Int64Index([885, 886, 887, 888, 889, 890, 891, 892, 893, 894, 895, 896, 897,
            898, 899, 900, 901, 902],
           dtype='int64')

In [ ]:
torch.lo

In [87]:
accuracy_scores_random = []

for j in range(10):
    predictions_results = []
    for i,class_name in enumerate(labels_df['class'].unique()):
        temp = labels_df[labels_df['class']==class_name]
        lenght = temp.shape[0]
        val = np.random.randint(lenght)
        cur_idx = temp.index[val]
        D,I = nbrs.kneighbors(embeddings[cur_idx].reshape([1,-1]))
        print(D,I)
        dft = pd.DataFrame(np.concatenate([D,I]).T,columns=['D','I'])
        dft['label'] = labels[dft['I'].astype(int)]
        pred = dft.groupby('label').mean().sort_values('D').index[0]
        real = temp.iloc[val]['class']
        #pred = df.iloc[I.flatten()].groupby('class').count().sort_values('image',ascending=False).iloc[0].name
        predictions_results.append(real==pred)
    df_pred = pd.DataFrame(predictions_results,columns=['result'])
    accuracy_scores_random.append(df_pred.value_counts()[True]/df_pred.shape[0])

In [88]:
accuracy_scores_random

[0.9910313901345291,
 0.9955156950672646,
 0.9955156950672646,
 0.968609865470852,
 0.9865470852017937,
 0.9955156950672646,
 1.0,
 0.9955156950672646,
 0.9955156950672646,
 0.9910313901345291]

In [78]:
temp.iloc[val]

class              ABC
image    IMG_1520.HEIC
Name: 24, dtype: object

In [74]:
cur_idx

24

In [64]:
df.iloc[I.flatten()].groupby('class').count().sort_values('image',ascending=False)

,image
class,
Tir,9


In [61]:
predictions_results

[True]

In [86]:
with open(os.path.join(BASE_PATH,'list_numpy.pkl'),'wb') as f:
    pickle.dump(full_list,f)